### **Cos'è LangChain?**
> LangChain è un framework per lo sviluppo di applicazioni che sfruttano modelli di linguaggio.

**TLDR**: LangChain rende più semplici le parti complicate di utilizzo e costruzione di modelli di intelligenza artificiale. Ciò avviene in due modi:

1. **Integration** - Importa dati esterni, come i filed, altre applicazioni o dati da API, ai tuoi modelli di linguaggio.
2. **Agentività** - Consente ai modelli di linguaggio di interagire con l'ambiente prendendo decisioni e utilizzando tools esterni. Utilizza i modelli di linguaggio per aiutarti a decidere quale azione intraprendere successivamente.

### **Perché LangChain?**
1. **Components** - LangChain facilita la sostituzione e l'astrazione dei componenti necessari per lavorare con modelli di linguaggio.

2. **Chain personalizzate** - LangChain fornisce supporto integrato per l'utilizzo e la personalizzazione di 'chain' - una serie di azioni concatenate.

3. **Velocità 🚢** - Il team è incredibilmente veloce nel rilascio. Sarai sempre aggiornato sulle ultime funzionalità dei modelli di linguaggio.

4. **Community 👥** - Supporto tramite Discord e comunità, incontri, hackathon, ecc.


In [1]:
# Load .env file
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path="../.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_api_key=OPENAI_API_KEY

# Componenti LangChain

## Schema di input
### **Text**
Il modo di interagire con i modelli utilizzando il linguaggio naturale


In [2]:
# Possiamo lavorare con testi semplici
my_text = "Che giorno viene dopo Venerdì?"

### **Chat Messages**
Simili al testo, ma vengo associati con un tipo di messaggio (System, Human, AI)

* **System** - Contesto di sfondo utile che indica all'AI cosa fare
* **Human** - Messaggi che intendono rappresentare l'utente
* **AI** - Messaggi che mostrano ciò a cui l'AI ha risposto

Per ulteriori informazioni, consulta la [documentazione](https://platform.openai.com/docs/guides/chat/introduction) di OpenAI.


In [3]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

history = [
    SystemMessage(content="You are an helpful chatbot that helps a user to remember the days of the week"),
    HumanMessage(content=my_text),
]

### **Documents**
Un oggetto che contiene un stringa di testo e dei metadati (ulteriori informazioni su quel testo)

In [4]:
from langchain.schema import Document
Document(page_content="Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
         metadata={
             'id' : 42,
             'source' : "https://www.loremipsum.com",
             'create_time' : "2021-01-01 12:00:00",
         })

Document(page_content='Lorem ipsum dolor sit amet, consectetur adipiscing elit.', metadata={'id': 42, 'source': 'https://www.loremipsum.com', 'create_time': '2021-01-01 12:00:00'})

## Models - The interface to the AI brains

###  **Language Model**
Un modello di linguaggio è un modello che prende in input una stringa di testo e restituisce una stringa di testo.

In [5]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=openai_api_key)

In [6]:
llm(my_text)

'\n\nSabato.'

### **Chat Model**
Un modello di chat è un modello che prende in input una serie di messaggi e restituisce un messaggio di output

In [7]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=1, openai_api_key=openai_api_key)

In [8]:
chat(history)

AIMessage(content='Dopo Venerdì viene Sabato.', additional_kwargs={}, example=False)

### **Text Embedding Model**
Un modello di embedding di testo è un modello che prende in input una stringa di testo e restituisce un vettore (una serie di numeri che rappresentano il significato semantico del testo).
Principalmente utilizzato quando si confrontano due stringe di testo insieme.

Vedi Word2Vec, sviluppata da Tomas Mikolov e il suo team presso Google nel 2013. [Paper](https://arxiv.org/pdf/1301.3781.pdf) e [video](https://www.youtube.com/watch?v=ERibwqs9p38).

Esempio vicinanza semantica:
![Word2Vec](../documents/Word-Vectors.webp)


In [9]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

text = "I'm playing with TransformErs models. They are so cool and powerful!"
tokens_integer=encoding.encode(text)
tokens_string = [encoding.decode_single_token_bytes(token) for token in tokens_integer]
tokens_string

[b'I',
 b"'m",
 b' playing',
 b' with',
 b' Transform',
 b'E',
 b'rs',
 b' models',
 b'.',
 b' They',
 b' are',
 b' so',
 b' cool',
 b' and',
 b' powerful',
 b'!']

OpenAI fornisce 3 tipi diversi di [modelli di embedding di testo](https://openai.com/blog/introducing-text-and-code-embeddings/):
- Text similarity
- Text search
- Code search

Recentemente sostiuti da uno all purpose: `text-embedding-ada-002`

In [10]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', openai_api_key=openai_api_key)

In [11]:
text = "Sono un appassionato di tecnologia e mi piace leggere libri di fantascienza."

In [12]:
text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

Your embedding is length 1536
Here's a sample: [-0.005055488087236881, -0.003664463059976697, 0.0002449230814818293, -0.032967910170555115, -0.026031168177723885]...


Proviamo a vedere la distanza del testo tra altre due stringhe di testo:

In [13]:
from scipy import spatial
text_embedding_1 = embeddings.embed_query('Io amo i libri di Frank Herbert.')
text_embedding_2 = embeddings.embed_query('Non riesco veramente a capire come installare Linux.')

result_1 = 1 - spatial.distance.cosine(text_embedding, text_embedding_1)
result_2 = 1 - spatial.distance.cosine(text_embedding, text_embedding_2)

print (f"Similarity between text and text_1: {result_1}")
print (f"Similarity between text and text_2: {result_2}")

Similarity between text and text_1: 0.8707046418153304
Similarity between text and text_2: 0.7741436811379714


## Prompts - Testo utilizzato per dare istruzioni al tuo modello

### **Prompt**
Cosa dirai al tuo modello.

In [14]:
from langchain.llms import OpenAI

llm = OpenAI( openai_api_key=openai_api_key)

prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement?
"""

llm(prompt)

'\nIt should say "Tomorrow is Tuesday."'

### **Prompt Template**
Un oggetto che aiuta a creare prompt basati su una combinazione di input utente, altre informazioni non statiche e una stringa di modello fissa.

Come se fosse una [f-string](https://realpython.com/python-f-strings/) in python ma per i prompt.

In [15]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Rome')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
I really want to travel to Rome. What should I do there?

Respond in one short sentence

-----------
LLM Output: Explore the Colosseum, the Pantheon, and the Roman Forum to experience the city's ancient history.


## Chains 

Un LLMChain è il tipo di catena più comune in LangChain. È composto da un PromptTemplate, un modello (LLM o ChatModel) e un parser di output opzionale. 

Cobinazione di diverse chiamate LLM e possibili azioni automatiche.

Ex: Summary #1, Summary #2, Summary #3 > Final Summary

Guarda [questo video](https://www.youtube.com/watch?v=f9_BWhCI4Zo&t=2s) che spiega i diversi tipi di summarization chains.

Esistono diverse [tipi di chains](https://python.langchain.com/en/latest/modules/chains/how_to_guides.html), cerca di vedere quali sono le migliori per il tuo caso d'uso. 

Noi ne useremo due:

### 1. Simple Sequential Chains

Chains semplici in cui l'output di un LLM viene utilizzato come input in un altro. Ottimo per suddividere i compiti (e mantenere l' LLM focalizzato).

In [16]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

In [17]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [18]:
# Proviamo la prima chain

location_chain.run("Rome")

'Spaghetti alla Carbonara, a delicious pasta dish originating in Rome, Italy. It is made with spaghetti noodles, egg, guanciale (cured pork jowl), pecorino cheese, and pepper.'

In [19]:
# Possibilità di fare batchin in input
input_list = [
    {"user_location": "Paris"},
    {"user_location": "Berlin"},
    {"user_location": "Lisbon"}
]

location_chain.apply(input_list)

[{'text': 'One classic dish from Paris is Coq au Vin, a French-style chicken, red wine, and mushroom dish.'},
 {'text': 'A classic dish from Berlin is the traditional Berliner Currywurst. It is a fast food dish made from a bratwurst sausage that is cut into pieces and served with curry ketchup.'},
 {'text': 'A classic dish from Lisbon would be bacalhau a bras, which is cod fish fried with potatoes, eggs, and onions.'}]

In [20]:
# Seconda chain

template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [21]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [22]:
review = overall_chain.run("Rome")



> Entering new  chain...

One classic dish from Rome is Spaghetti alla Carbonara, made with eggs, bacon, pecorino cheese, and black pepper.

Spaghetti alla Carbonara Recipe 
Ingredients:
- 8oz spaghetti 
- 4 strips bacon, cut into small cubes
- 2 eggs
- 1/2 cup grated pecorino cheese
- Freshly cracked black pepper, to taste 
- Olive oil

Instructions:
1. Start by cooking your spaghetti according to package directions.
2. In a skillet over medium heat, cook the bacon cubes until crisp.
3. In a separate bowl, beat together the eggs and pecorino cheese.
4. Drain spaghetti in a colander and add back to the pot.
5. Turn heat to low and add the egg and cheese mixture and stir constantly until creamy.
6. Add the bacon and black pepper, then drizzle with olive oil.
7. Serve while hot, and enjoy!

> Finished chain.


### 2. Summarization Chain
Chain già implementata che summarizza facilmente lunghi documenti e ne ottiene un riassunto. Guarda [questo video](https://www.youtube.com/watch?v=f9_BWhCI4Zo) per altri tipi di chain oltre alla map-reduce.

In [23]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('../documents/preface_fondazione_asimov.txt')
documents = loader.load()

# Prepara lo splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

# Dividi il documento in chunks
texts = text_splitter.split_documents(documents)

# Ci sta un po' di complessità in questa riga. Ti consiglio di guardare il video sopra per maggiori dettagli
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
res = chain.run(texts)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"Giuseppe Lippi
Le origini della Fondazione
Come è nato il più famoso ciclo fantascientifico
di tutti i tempi
Apparso sul n. 1203 di Urania (18 aprile 1993)
È il mattino del 1° agosto 1941; a New York, nella Settima Avenue, un giovanotto
di belle speranze sale i gradini del palazzo Street & Smith, la vecchia casa editrice
specializzata in pulp magazines che pubblica Astounding Science Fiction (ex
Astounding Stories). Il giovanotto ha un appuntamento col signor Campbell, sì, John
W. Campbell jr., conferma all’usciere mentre attende impaziente il pass.
Attraversa quindi alcuni corridoi, e finalmente (dopo la visione di una magica"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"stanza in cui sono accumulati, in tanti pacchetti, gli Astounding del mese dopo!),
accede all’ufficio del signor Campbell. Il giovanotto è Isaac Asimov

In [24]:
import pprint
pprint.pprint(res)

('\n'
 '\n'
 'In 1941, Isaac Asimov has an appointment with the editor of Astounding '
 'Science Fiction Magazine and accidentally finds inspiration for the story '
 'that becomes one of the most famous science-fiction series of all time - the '
 'Foundation series. Asimov uses Pyschostoria, a fictional science based on '
 'the laws of statistics as a central theme in his story and the series has '
 'since been highly praised as one of the greatest science fiction works of '
 'all time. The cycle ends with the publication of his last book ‘Foundation '
 'Anno Zero’ shortly before his death in 1992.')
